In [1]:
# 1. Preparing the text data

# The emails in Ling-spam corpus have been already preprocessed in the following ways:
# a) Removal of stop words
# b) Lemmatization
# We still need to remove the non-words like punctuation marks or special characters from the mail documents.

In [2]:
# 2. Creating word dictionary

# As a first step, we need to create a dictionary of words and their frequency.
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]    
    all_words = []       
    for mail in emails:    
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:  #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words
    
    dictionary = Counter(all_words)
    
    list_to_remove = list(dictionary)
    for item in list_to_remove:
        if item.isalpha() == False: 
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    
    return dictionary

In [3]:
# 3. Feature extraction process

def extract_features(mail_dir): 
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    docID = 0;
    for fil in files:
      with open(fil) as fi:
        for i,line in enumerate(fi):
          if i == 2:
            words = line.split()
            for word in words:
              wordID = 0
              for i,d in enumerate(dictionary):
                if d[0] == word:
                  wordID = i
                  features_matrix[docID,wordID] = words.count(word)
        docID = docID + 1     
    return features_matrix

In [4]:
# 4. Training the classifiers

import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
 
# Create a dictionary of words with its frequency

train_dir = 'train-mails'
dictionary = make_Dictionary(train_dir)

# Prepare feature vectors per training mail and its labels

train_labels = np.zeros(702)
train_labels[351:701] = 1
train_matrix = extract_features(train_dir)

# Training SVM and Naive Bayes classifier and its variants

model1 = LinearSVC()
model2 = MultinomialNB()

model1.fit(train_matrix,train_labels)
model2.fit(train_matrix,train_labels)

# Test the unseen mails for Spam

test_dir = 'test-mails'
test_matrix = extract_features(test_dir)
test_labels = np.zeros(260)
test_labels[130:260] = 1

result1 = model1.predict(test_matrix)
result2 = model2.predict(test_matrix)

print (confusion_matrix(test_labels,result1))
print (confusion_matrix(test_labels,result2))

[[126   4]
 [  6 124]]
[[129   1]
 [  9 121]]


In [5]:
# 4. Predicting with SVM

predict_dir = 'predict-mail'
predict_matrix = extract_features(predict_dir)
predict_result = model1.predict(predict_matrix)
# 0: Ham / 1: Spam
print (predict_result)

[ 0.]


In [6]:
# 5. Predicting with Naive Bayes

predict_dir = 'predict-mail'
predict_matrix = extract_features(predict_dir)
predict_result = model2.predict(predict_matrix)
# 0: Ham / 1: Spam
print (predict_result)

[ 0.]
